---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from scipy import stats

#%matplotlib notebook
#import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    labels = []
    for graph in P1_Graphs:
        #path = nx.average_shortest_path_length(graph)
        clstr = nx.average_clustering(graph)
        degree = nx.degree(graph)
        max_degree = max(set(degree.values()))
        
        if max_degree > 80:
            labels.append('PA')
        elif clstr < .1:
                labels.append('SW_H')
        else:
            labels.append('SW_L')
    return labels

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
#nx.draw_networkx(G, node_size=10, width=.1, with_labels=False)

In [7]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import cross_val_score

#lr = LogisticRegression().fit(X_train, y_train)
#grid_values = {'penalty': ['l1', 'l2'], 'C':[0.01, 0.1, 1, 10, 100]}

#grid_lr_rec = GridSearchCV(lr, param_grid = grid_values, scoring = 'roc_auc', cv=3)
#grid_lr_rec.fit(X_train, y_train)

#scores = grid_lr_rec.cv_results_['mean_test_score'].reshape(5,2) 
#scores

In [8]:
def GridSearch_Heatmap(scores):
    %matplotlib notebook
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.figure()
    sns.heatmap(scores.reshape(5,2), xticklabels=['l1','l2'], yticklabels=[0.01, 0.1, 1, 10, 100])
    plt.yticks(rotation=0);
    
#GridSearch_Heatmap(scores)

In [9]:
def salary_predictions():
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score

    #Initialize DataFrame
    df = pd.DataFrame(index=G.nodes())

    #Get Attributes
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    #Make Features - page rank, clustering, betweenness, closeness, deg centrality
    #df['Degree'] = pd.Series(nx.degree(G))
    df['DegreeCentrality'] = pd.Series(nx.degree_centrality(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Betweenness'] = pd.Series(nx.betweenness_centrality(G))
    df['Closeness'] = pd.Series(nx.closeness_centrality(G))
    
    features = ['PageRank', 'DegreeCentrality', 'Closeness', 'Clustering', 'Betweenness']

    #Training Set (Drop if NaN in ManagementSalary) 
    train = df.dropna(subset = ['ManagementSalary'])
    X_train = train[features]
    y_train = train['ManagementSalary']

    #Test Set
    test = df.loc[df['ManagementSalary'].isnull()]
    X_test = test[features]
    
    #Logistic Regression
    lr = LogisticRegression(C=100).fit(X_train, y_train)

    #lr.predict(X_test)
    pred = pd.Series(lr.predict_proba(X_test)[:,1], index=X_test.index)
    
    
    
    return pred

In [10]:
salary_predictions()

1       0.104712
2       0.567128
5       0.911301
8       0.147733
14      0.237042
18      0.233739
27      0.256807
30      0.413254
31      0.179957
34      0.137257
37      0.066708
40      0.155933
45      0.152177
54      0.300218
55      0.325449
60      0.130413
62      0.983985
65      0.386212
77      0.079034
79      0.163481
97      0.083727
101     0.038574
103     0.256630
108     0.122142
113     0.496814
122     0.019812
141     0.337367
142     0.881761
144     0.071059
145     0.537800
          ...   
913     0.020250
914     0.058047
915     0.000104
918     0.148572
923     0.008686
926     0.072060
931     0.034823
934     0.003794
939     0.002119
944     0.000145
945     0.014466
947     0.149934
950     0.107546
951     0.013016
953     0.010682
959     0.000267
962     0.000137
963     0.156614
968     0.066965
969     0.068317
974     0.085815
984     0.003110
987     0.099703
989     0.073045
991     0.083498
992     0.000491
994     0.000311
996     0.0001

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [11]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [12]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import cross_val_score

#lr = LogisticRegression().fit(X_train, y_train)
#grid_values = {'penalty': ['l1', 'l2'], 'C':[0.01, 0.1, 1, 10, 100]}

#grid_lr_rec = GridSearchCV(lr, param_grid = grid_values, scoring = 'roc_auc', cv=3)
#grid_lr_rec.fit(X_train, y_train)

#scores = grid_lr_rec.cv_results_['mean_test_score'].reshape(5,2) 
#scores

In [15]:
def new_connections_predictions():
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    
    #Initialize DataFrame
    df = pd.DataFrame(index=G.edges())

    #Edge Features
    future_connections['Preferential Attachment'] = [i[2] for i in (nx.preferential_attachment(G, ebunch=future_connections.index))]
    future_connections['Common Neighbors'] = future_connections.index.map(lambda employee: len(list(nx.common_neighbors(G, employee[0], employee[1]))))
    features = ['Common Neighbors']
    
    #Find out which edge pairs are missing data and separate into train/test data
    test = future_connections.loc[future_connections['Future Connection'].isnull()]
    train = future_connections.loc[~future_connections['Future Connection'].isnull()]

    X_train = train[features]
    y_train = train['Future Connection']

    X_test = test[features]
    
    #Logistic Regression
    lr = LogisticRegression().fit(X_train, y_train)

    #lr.predict(X_test)
    pred = pd.Series(lr.predict_proba(X_test)[:,1], index=X_test.index)
    
    return pred

In [16]:
new_connections_predictions()

(107, 348)    0.036339
(542, 751)    0.013647
(20, 426)     0.675424
(50, 989)     0.013647
(942, 986)    0.013647
(324, 857)    0.013647
(13, 710)     0.218828
(19, 271)     0.218828
(319, 878)    0.013647
(659, 707)    0.013647
(49, 843)     0.013647
(208, 893)    0.013647
(377, 469)    0.013647
(405, 999)    0.022332
(129, 740)    0.022332
(292, 618)    0.022332
(239, 689)    0.013647
(359, 373)    0.013647
(53, 523)     0.036339
(276, 984)    0.013647
(202, 997)    0.013647
(604, 619)    0.036339
(270, 911)    0.013647
(261, 481)    0.093200
(200, 450)    0.774545
(213, 634)    0.013647
(644, 735)    0.036339
(346, 553)    0.013647
(521, 738)    0.013647
(422, 953)    0.022332
                ...   
(672, 848)    0.013647
(28, 127)     0.939241
(202, 661)    0.013647
(54, 195)     0.999290
(295, 864)    0.013647
(814, 936)    0.013647
(839, 874)    0.013647
(139, 843)    0.013647
(461, 544)    0.013647
(68, 487)     0.013647
(622, 932)    0.013647
(504, 936)    0.022332
(479, 528) 